In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%late

In [2]:
!git clone https://github.com/nothings/stb.git
!ls
!mkdir TestCurl
!mv /content/stb/stb_image*.* /content/TestCurl

Cloning into 'stb'...
remote: Enumerating objects: 7851, done.
remote: Counting objects: 100% (580/580), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 7851 (delta 376), reused 547 (delta 365), pack-reused 7271
Receiving objects: 100% (7851/7851), 5.45 MiB | 19.53 MiB/s, done.
Resolving deltas: 100% (5221/5221), done.
sample_data  stb


In [3]:
!ls

sample_data  stb  TestCurl


In [4]:
%%writefile TestCurl/TestCurl.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <curl/curl.h>

struct MemoryStruct {
  char *memory;
  size_t size;
};

size_t WriteMemoryCallback(void *contents, size_t size, size_t nmemb, void *userp);

int main(int argc, char *argv[]) {
  if( argc != 2 ) {
    printf("usage: try './curl [url]' to make a get request.\n");
    return 1;
  }

  CURL *curl_handle;
  CURLcode res;

  struct MemoryStruct chunk;
  chunk.memory = malloc(1);  
  chunk.size = 0;

  curl_handle = curl_easy_init();
  if(curl_handle) {
    curl_easy_setopt(curl_handle, CURLOPT_URL, argv[1]);
    curl_easy_setopt(curl_handle, CURLOPT_FOLLOWLOCATION, 1L);
    curl_easy_setopt(curl_handle, CURLOPT_WRITEFUNCTION, WriteMemoryCallback);
    curl_easy_setopt(curl_handle, CURLOPT_WRITEDATA, (void *)&chunk);
    curl_easy_setopt(curl_handle, CURLOPT_USERAGENT, "libcurl-agent/1.0");

    res = curl_easy_perform(curl_handle);

    if(res != CURLE_OK) {
      fprintf(stderr, "error: %s\n", curl_easy_strerror(res));
    } else {
      printf("Size: %lu\n", (unsigned long)chunk.size);
      printf("Data: %s\n", chunk.memory);
    }
    curl_easy_cleanup(curl_handle);
    free(chunk.memory);
  }
  return 0;
}

size_t WriteMemoryCallback(void *contents, size_t size, size_t nmemb, void *userp) {
  size_t realsize = size * nmemb;
  struct MemoryStruct *mem = (struct MemoryStruct *)userp;

  char *ptr = realloc(mem->memory, mem->size + realsize + 1);
  if(ptr == NULL) {
    printf("error: not enough memory\n");
    return 0;
  }

  mem->memory = ptr;
  memcpy(&(mem->memory[mem->size]), contents, realsize);
  mem->size += realsize;
  mem->memory[mem->size] = 0;

  return realsize;
}

Writing TestCurl/TestCurl.c


In [5]:
!ls -l

total 12
drwxr-xr-x 1 root root 4096 Sep 22 13:42 sample_data
drwxr-xr-x 9 root root 4096 Sep 26 19:20 stb
drwxr-xr-x 2 root root 4096 Sep 26 19:21 TestCurl


In [6]:
%%shell
cd TestCurl
gcc -Wall TestCurl.c -o TestCurl -l curl -lm


In [7]:
!ls

sample_data  stb  TestCurl


In [10]:
!./TestCurl/TestCurl www.cteq.eu

Size: 2379
Data: <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<!-- saved from url=(0026)http://labviewnxt.cteq.eu/ -->
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">  

<title>: cteq.eu</title>
<meta name="keywords" content="Embedded systems, Electronics, Innovation, Python, NVIDIA GPU, Xilinx, Acceleration, Zynq, GPU"> 
<meta name="description" content="cteq.eu: Sea of Technology">
  
<style type="text/css" media="screen"><!--
body 
    {
    color: white;
    background-color: #FF;
    margin: 0px
    }

#horizon        
    {
    color: black;
    background-color: transparent;
    text-align: left;
    position: absolute;
    top: 45%;
    left: 0px;
    width: 100%;
    height: 1px;
    overflow: visible;
    visibility: visible;
    display: block
    }

#content    
    {
    font-family: BankGothic Md BT,Verdana, Geneva, Arial, sans-serif;
    background-color: transparent;
    margin-left: -125px;
    position: absolute;
    

In [13]:
%%writefile TestCurl/TestCurlImage.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <curl/curl.h>

static size_t write_data(void *ptr, size_t size, size_t nmemb, void *stream)
{
  size_t written = fwrite(ptr, size, nmemb, (FILE *)stream);
  return written;
}

int main(int argc, char *argv[]) {
  CURL *curl_handle;
  static const char *ImageFileName = "ImageFileName.png";
  FILE *Imagefile;
 
  if(argc < 2) {
    printf("Usage: %s <URL>\n", argv[0]);
    return 1;
  }
 
  curl_global_init(CURL_GLOBAL_ALL);
 
  /* init the curl session */
  curl_handle = curl_easy_init();
 
  /* set URL to get here */
  curl_easy_setopt(curl_handle, CURLOPT_URL, argv[1]);
 
  /* Switch on full protocol/debug output while testing */
  curl_easy_setopt(curl_handle, CURLOPT_VERBOSE, 1L);
 
  /* disable progress meter, set to 0L to enable it */
  curl_easy_setopt(curl_handle, CURLOPT_NOPROGRESS, 1L);
 
  /* send all data to this function  */
  curl_easy_setopt(curl_handle, CURLOPT_WRITEFUNCTION, write_data);
 
  /* open the file */
  Imagefile = fopen(ImageFileName, "wb");
  if(Imagefile) {
 
    /* write the page body to this file handle */
    curl_easy_setopt(curl_handle, CURLOPT_WRITEDATA, Imagefile);
 
    /* get it! */
    curl_easy_perform(curl_handle);
 
    /* close the header file */
    fclose(Imagefile);
  }
 
  /* cleanup curl stuff */
  curl_easy_cleanup(curl_handle);
 
  curl_global_cleanup();
 
  return 0;
}


Overwriting TestCurl/TestCurlImage.c


In [14]:
%%shell
cd TestCurl
gcc -Wall TestCurlImage.c -o TestCurlImage -l curl -lm

In [16]:
!./TestCurl/TestCurlImage "https://www.pxl.be/Assets/website/pxl_algemeen/afbeeldingen/grotere_versie/logo_pxl_digital.png"

*   Trying 193.190.154.243...
* TCP_NODELAY set
* Connected to www.pxl.be (193.190.154.243) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
* SSL connection using TLSv1.2 / DHE-RSA-AES128-GCM-SHA256
* ALPN, server did not agree to a protocol
* Server certificate:
*  subject: C=BE; ST=Limburg; O=Hogeschool PXL; CN=*.pxl.be
*  start date: May  2 00:00:00 2022 GMT
*  expire date: May  2 23:59:59 2023 GMT
*  subjectAltName: host "www.pxl.be" matched cert's "*.pxl.be"
*  issuer: C=NL; O=GEANT Vereniging; CN=GEANT OV RSA CA 4
*  SSL certificate verify ok.
> GET /Assets/website/pxl_algemeen/afbeeldingen/grotere_versie/logo_pxl_digital.png HTTP/1.1
Host: www.pxl.be
Accept: */*

< HTTP/1.1 200 OK
< Content-Type: image/png
< Last-Modified: Wed, 02 Feb 2022 13:45:30 GMT
< Accept-Ranges: bytes
< ETag: "4ec49c243b18d81:0"
< Set-Cookie: PXLSite_guid=c7f5b32a-2465-4b57-947c-0f040cff22c2; e